# 对话式 RAG - Conversational RAG
在许多问答应用程序中，我们希望允许用户进行来回对话，这意味着应用程序需要某种过去问题和答案的“记忆”，以及将这些问题和答案纳入当前思维的逻辑。

在本指南中，我们重点介绍如何添加逻辑来整合历史消息。此处介绍了有关聊天记录管理的更多详细信息。

我们将介绍两种方法：

 - 链 Chains，我们始终在其中执行检索步骤；

 - 代理 Agents，我们让 LLM 自行决定是否以及如何执行检索步骤（或多个步骤）。
   

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [4]:
import bs4
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import BaichuanTextEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

loader = WebBaseLoader(
    web_paths=("https://ouzhoubei.co/article-794-1.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id=("article_content"))
    )
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vectorstore.as_retriever()

system_prompt = (
    "您是问答任务的助手。"
    "使用以下检索到的上下文来回答问题。如果您不知道答案，请说您不知道。最多使用三句话并保持答案简洁。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [5]:
response = rag_chain.invoke({"input": "意大利队有多少位球员?"})
response["answer"]

'意大利队有26位球员。'